<a href="https://colab.research.google.com/github/priya170807/text_analysis/blob/master/datacleaning_flipkart_version7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import re
import sys
import pickle
import pandas as pd
import numpy as np
from collections import Counter

In [3]:
df = pd.read_csv("/content/drive/MyDrive/datasets/flipkart_com-ecommerce_sample.csv")

In [4]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [5]:
useful_data = df[["product_name","product_category_tree", "description", "brand"]]

In [6]:
useful_data.head()

,product_name,product_category_tree,description,brand
0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Alisha
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,AW
3,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Alisha
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,Sicons


In [7]:
len(useful_data)

20000

In [8]:
data = useful_data.copy()

Remove all the non-word characters and also the " ' " between the letters in the product_name.

In [9]:
# lets clean the product_name column
data["product_name"] = data["product_name"].str.replace(r'(?<=[a-zA-Z\d\s])\'(?=[a-zA-Z\d\s])', r"", regex=True)

In [10]:
data["product_name"] = data["product_name"].str.replace(r'[^a-zA-Z\d\s]', r"", regex=True)

In [11]:
data["product_name"] = data["product_name"].str.replace(r'Dr.', r'Dr',regex=True)

In [12]:
# randomly check the product_name column 
data["product_name"][12]

'Sicons All Purpose Tea Tree Dog Shampoo'

In [13]:
data["product_name"][440]

'Asian Gems  Jewels 5 Dhaatu Metal Alloy Zircon Ring'

In [14]:
data["product_name"][339]

'Drssberry Black Synthetic  Clutch'

Clean the product_category_tree column.

In [15]:
# create new column "mainsnak" which is same as data["product_category_tree"]
data["mainsnak"] = data["product_category_tree"]

In [16]:
data["mainsnak"][0]

'["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'

In [17]:
# we can see a single quotes occuring in the mainsnak column
data["mainsnak"][57]

'["MASARA Solid Women\'s Straight Kurta"]'

In [18]:
# create a function to split the rows in data["product_category_tree"] and get the first string or the string at zeroth position.
def get_instance_of_entries(entry):
  entry_list = entry.split(">>")
  first_word = entry_list[0].rstrip()
  first_word = re.sub(r'[^a-zA-Z\d\s]', r'', first_word)
  first_word = re.sub(r'Dr.', r'Dr', first_word)
  # first_word = re.sub(r'[["]', r'', first_word)
  # first_word = re.sub(r'["]]', r'', first_word)
  return first_word

In [19]:
# apply the function get_instance_of_entries to mainsnak column and get the string in zeroth position. 
data["mainsnak"] = data["mainsnak"].map(lambda x : get_instance_of_entries(x))

In [20]:
data["mainsnak"][57]

'MASARA Solid Womens Straight Kurta'

In [21]:
data["mainsnak"][65]

'Libas Printed Womens Anarkali Kurta'

Remove all the non-word characters and also the " ' " between the letters in the column brand.

In [22]:
# lets clean the brand column
# replace the single quotes by empty space
data["brand"] = data["brand"].str.replace(r'(?<=[a-zA-Z\d\s])\'(?=[a-zA-Z\d\s])', r"", regex=True)

In [23]:
data["brand"] = data["brand"].str.replace(r'[^a-zA-Z\d\s]', r"", regex=True)

In [24]:
data["brand"] = data["brand"].str.replace(r'Dr.', r'Dr',regex=True)

In [25]:
data["brand"][5357]

'Mi Dulce Anya'

In [26]:
data["brand"][19289]

'Cynthias Fashion'

In [27]:
data["brand"][19120]

'Layerr'

check the data["description"] column and start cleaning it

In [28]:
# creating a new column data["new_desc"] from data["description"]
data["new_desc"] = data["description"]

In [29]:
# removing the Rs. from the data since our end-token is the '.'
data["new_desc"] = data["new_desc"].str.replace(r'Rs.', '',regex=True)

In [30]:
data["new_desc"] = data["new_desc"].str.replace(r'Dr.', r'Dr',regex=True)

In [31]:
data["new_desc"][1]

"FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Price:  22,646 • Fine deep seating experience • Save Space with the all new click clack Sofa Bed • Easy to fold and vice versa with simple click clack mechanism • Chrome legs with mango wood frame for long term durability • Double cushioned Sofa Bed to provide you with extra softness to make a fine seating experience • A double bed that can easily sleep two,Specifications of FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Installation & Demo Installation & Demo Details Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product. In The Box 1 Sofa Bed General Brand FabHomeDecor Mattress Included No Delivery Condition Knock Down Storage Included No Mechanism Type Pull Out Type Sofa Bed Style Contemporary & Modern Fil

In [32]:
# replace the bullets with period.
data["new_desc"] = data["new_desc"].str.replace(r'\u2022+?',r'.',regex=True)

In [33]:
# replace the comma between the words with nothing(empty space)
data["new_desc"] = data["new_desc"].str.replace(r'(?<=\d),(?=\d)', r'',regex=True)

In [34]:
data["new_desc"][1]

"FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Price:  22646 . Fine deep seating experience . Save Space with the all new click clack Sofa Bed . Easy to fold and vice versa with simple click clack mechanism . Chrome legs with mango wood frame for long term durability . Double cushioned Sofa Bed to provide you with extra softness to make a fine seating experience . A double bed that can easily sleep two,Specifications of FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Installation & Demo Installation & Demo Details Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product. In The Box 1 Sofa Bed General Brand FabHomeDecor Mattress Included No Delivery Condition Knock Down Storage Included No Mechanism Type Pull Out Type Sofa Bed Style Contemporary & Modern Fill

In [35]:
data["new_desc"][0]

"Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"

In [36]:
data["new_desc"][439]

"Asian Gems & Jewels 6.25 Ratti 5 Dhaatu Sterling Silver, Metal, Alloy Cat's Eye NA K Ring\n                         Price:  828\n\t\t\t\t\n\t\t\tAsian Gem & Jewellery offers you Panch dhaatu Cultured Gemstone Finger Ring . 'ASIAN GEMS & JEWELS' and all its grading standards and content (Copyright Registered)\n belongs to the Owners of NEW INDIA WORKS (NIW).\nIn Hindi, '9' means 'Nav' and '5' means 'Punch'. Gem is called 'Ratna', whereas any 'Metal or Mineral or Element' is referred as 'Dhaatu'.\nRegarding 9 Gemstones. 'Nav Ratnas' means an 'Astro Planetary' communicating team of 9 Precious and Semi Precious Gemstones those are believed to reach and connect out to 9 Planets residing, revolving in our galaxy. \nIn the Birth Horoscope of an individual, if any planet is weak or some other planets are not sitting in the right degree/position or house or are opposing or suppressing some other planets than :-\nRUBY (MANIK) is considered to connect, enhance and empower Planet Sun (Surya) . \n

In [37]:
# this is a regex creating a space after : or , or ; or ( or - when the immediate character that follows is not space.
data["new_desc"] = data["new_desc"].str.replace(r'(?<=[:;,(-])(?=[^\s])', r' ',regex=True)

In [38]:
# this is a regex creates a space for every word character that is followed by any of the non-word charcter defined in the positive lookahead group ([!):;,-])
data["new_desc"] = data["new_desc"].str.replace(r'(?<=[\w])([!):;,-])', r' \1',regex=True)

In [39]:
# this is a regex creates a space for every word character or non-word character that is followed by one or more period(non-word charcter) 
# defined in the positive lookahead group (\.+)
data["new_desc"] = data["new_desc"].str.replace(r'(?<=[\w\W])([\.+])', r' ',regex=True)

In [40]:
# this sis a regex that creates a single space in place of multiple spaces
data["new_desc"] = data["new_desc"].str.replace(r'\s{2,}', r' ',regex=True)

In [41]:
# this is a regex creates a space for every word character or non-word character that is followed by one or more newline (non-word charcter) 
# defined in the positive lookahead group (\n+)
data["new_desc"] = data["new_desc"].str.replace(r'(?<=[\w\W])([\n+])', r' ',regex=True)

In [42]:
# the regex for replacing single commas by no space charcater
data["new_desc"] = data["new_desc"].str.replace(r',', r'', regex=True)

In [43]:
# According to the text since the colon and the semicolon characters formed a meaningful sentence end, these characters are replaced by period.
# This is a regex to replace the colon(:) and the semicolon(;) by the period(.)
data["new_desc"] = data["new_desc"].str.replace(r'[:;]', r'.', regex=True)

In [44]:
data["new_desc"][5357]

"Specifications of Mi Dulce An'ya Solid Single Breasted Formal Boy's Blazer General Details Ideal For Boy's Occasion Formal Pattern Solid Blazer Details Type Single Breasted Fabric Organic Cotton Sleeve Full Sleeve Closure Button Pockets 2 Front Slip Pockets Vents Vent at back Lining Cotton Lining Number of Contents in Sales Package Pack of 1 Fabric Care Gentle Wash With Similar Colors Additional Details Style Code SS15 - IB - WC001A In the Box Blazer"

In [45]:
# regex to remove the single quotes between the word characters in a given text
data["new_desc"] =  data["new_desc"].str.replace(r'(?<=[a-zA-Z\d\s])\'(?=[a-zA-Z\d\s])', r"", regex=True)

In [46]:
# check few examples after cleaning the data.
data["new_desc"][93]

'Vizion 8503B - 1RED Cold Light Digital Watch - For Boys  Girls Price . 595 Brand . Vizion  Sports Series watch for Kids  1 year manufacturer warranty  Digital display  Automatic movement  Metal Case and solid body  Dial shape . Round  Dial dimension . 22mm  Strap material . Silicone  Strap width . 20mm  Water resistant  Night display cold backLight  Daily Alarm  Hour minute second  date week display  12/24 hour format  Attractive brand packaging - Tuff plastic box  Note . In case of non - receipt of warranty card  please intimate us within 48 hours  Disclaimer . Product color may slightly vary due to photographic lighting sources or your monitor settings ( The image has been enlarged for better viewing ) Brand . Vizion  Sports Series watch for Kids  1 year manufacturer warranty  Digital display  Automatic movement  Metal Case and solid body  Dial shape . Round  Dial dimension . 22mm  Strap material . Silicone  Strap width . 20mm  Water resistant  Night display cold backLight  Daily Al

In [47]:
data["new_desc"][0]

'Key Features of Alisha Solid Womens Cycling Shorts Cotton Lycra Navy  Red  Navy  Specifications of Alisha Solid Womens Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Womens Fabric Care Gentle Machine Wash in Lukewarm Water  Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts'

In [48]:
data["new_desc"][2]

'Key Features of AW Bellies Sandals Wedges Heel Casuals  AW Bellies Price . 499 Material . Synthetic Lifestyle . Casual Heel Type . Wedge Warranty Type . Manufacturer Product Warranty against manufacturing defects . 30 days Care instructions . Allow your pair of shoes to air and de - odorize at regular basis . use shoe bags to prevent any stains or mildew . dust any dry dirt from the surface using a clean cloth . do not use polish or shiner  Specifications of AW Bellies General Ideal For Women Occasion Casual Shoe Details Color Red Outer Material Patent Leather Heel Height 1 inch Number of Contents in Sales Package Pack of 1 In the Box One Pair Of Shoes'

In [49]:
data["new_desc"][439]

'Asian Gems & Jewels 6 25 Ratti 5 Dhaatu Sterling Silver  Metal  Alloy Cats Eye NA K Ring Price . 828 Asian Gem & Jewellery offers you Panch dhaatu Cultured Gemstone Finger Ring ASIAN GEMS & JEWELS and all its grading standards and content ( Copyright Registered ) belongs to the Owners of NEW INDIA WORKS ( NIW ) In Hindi  9 means Nav and 5 means Punch Gem is called Ratna whereas any Metal or Mineral or Element is referred as Dhaatu Regarding 9 Gemstones Nav Ratnas means an Astro Planetary communicating team of 9 Precious and Semi Precious Gemstones those are believed to reach and connect out to 9 Planets residing  revolving in our galaxy In the Birth Horoscope of an individual  if any planet is weak or some other planets are not sitting in the right degree/position or house or are opposing or suppressing some other planets than . - RUBY ( MANIK ) is considered to connect  enhance and empower Planet Sun ( Surya ) BLUE SAPPHIRE ( NEELAM ) is considered to connect  enhance and empower Pla

In [50]:
data["new_desc"][5357]

'Specifications of Mi Dulce Anya Solid Single Breasted Formal Boys Blazer General Details Ideal For Boys Occasion Formal Pattern Solid Blazer Details Type Single Breasted Fabric Organic Cotton Sleeve Full Sleeve Closure Button Pockets 2 Front Slip Pockets Vents Vent at back Lining Cotton Lining Number of Contents in Sales Package Pack of 1 Fabric Care Gentle Wash With Similar Colors Additional Details Style Code SS15 - IB - WC001A In the Box Blazer'

In [51]:
# create a function that splits the data in each row of the data["new_desc"] series by (.)
data["new_desc"] = data["new_desc"].map(lambda x : str(x).split('.'))

In [52]:
data["new_desc"][5357]

['Specifications of Mi Dulce Anya Solid Single Breasted Formal Boys Blazer General Details Ideal For Boys Occasion Formal Pattern Solid Blazer Details Type Single Breasted Fabric Organic Cotton Sleeve Full Sleeve Closure Button Pockets 2 Front Slip Pockets Vents Vent at back Lining Cotton Lining Number of Contents in Sales Package Pack of 1 Fabric Care Gentle Wash With Similar Colors Additional Details Style Code SS15 - IB - WC001A In the Box Blazer']

In [53]:
def make_list(entry):
  temp_list=[]
  for string in entry:
    string_list = [item for item in string.split(" ") if not item == ""]
    temp_list.append(string_list)
  return temp_list

In [54]:
# create a function that splits the data in each row of the data["new_desc"] series by (" ").
data["new_desc"] = data["new_desc"].map(lambda x : make_list(x))

In [55]:
def append_period(entry):
  period = "."
  for lt in entry:
    lt.append(period)
  return entry
  

In [56]:
# create a function that appends each list in each row of the data["new_desc"] series by (.), which shows the end token for each sentence.
data["new_desc"] = data["new_desc"].map(lambda x : append_period(x))

In [57]:
# data["new_desc"][5357]

In [58]:
# check the data after transformations.
data.head()

,product_name,product_category_tree,description,brand,mainsnak,new_desc
0,Alisha Solid Womens Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Alisha,Clothing,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,Furniture,"[[FabHomeDecor, Fabric, Double, Sofa, Bed, (, ..."
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,AW,Footwear,"[[Key, Features, of, AW, Bellies, Sandals, Wed..."
3,Alisha Solid Womens Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Alisha,Clothing,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,Sicons,Pet Supplies,"[[Specifications, of, Sicons, All, Purpose, Ar..."


In [59]:
# create new dataframe with useful columns.
new_df = data[["product_name", "mainsnak", "brand", "new_desc"]]

In [60]:
new_df.head()

,product_name,mainsnak,brand,new_desc
0,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
1,FabHomeDecor Fabric Double Sofa Bed,Furniture,FabHomeDecor,"[[FabHomeDecor, Fabric, Double, Sofa, Bed, (, ..."
2,AW Bellies,Footwear,AW,"[[Key, Features, of, AW, Bellies, Sandals, Wed..."
3,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
4,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies,Sicons,"[[Specifications, of, Sicons, All, Purpose, Ar..."


In [61]:
# we observe that each row in the column "new_desc" is a list of list and each inner list is ended by a "." entry
# new_df["new_desc"][0]

In [62]:
new_df["new_desc"][5357]

[['Specifications',
  'of',
  'Mi',
  'Dulce',
  'Anya',
  'Solid',
  'Single',
  'Breasted',
  'Formal',
  'Boys',
  'Blazer',
  'General',
  'Details',
  'Ideal',
  'For',
  'Boys',
  'Occasion',
  'Formal',
  'Pattern',
  'Solid',
  'Blazer',
  'Details',
  'Type',
  'Single',
  'Breasted',
  'Fabric',
  'Organic',
  'Cotton',
  'Sleeve',
  'Full',
  'Sleeve',
  'Closure',
  'Button',
  'Pockets',
  '2',
  'Front',
  'Slip',
  'Pockets',
  'Vents',
  'Vent',
  'at',
  'back',
  'Lining',
  'Cotton',
  'Lining',
  'Number',
  'of',
  'Contents',
  'in',
  'Sales',
  'Package',
  'Pack',
  'of',
  '1',
  'Fabric',
  'Care',
  'Gentle',
  'Wash',
  'With',
  'Similar',
  'Colors',
  'Additional',
  'Details',
  'Style',
  'Code',
  'SS15',
  '-',
  'IB',
  '-',
  'WC001A',
  'In',
  'the',
  'Box',
  'Blazer',
  '.']]

In [63]:
len(new_df)

20000

In [64]:
# change the column names matching to the wikidataset column names.
new_df.columns = ['Name_ID', 'mainsnak', 'brand', 'TEXT']

In [65]:
new_df.head()

,Name_ID,mainsnak,brand,TEXT
0,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
1,FabHomeDecor Fabric Double Sofa Bed,Furniture,FabHomeDecor,"[[FabHomeDecor, Fabric, Double, Sofa, Bed, (, ..."
2,AW Bellies,Footwear,AW,"[[Key, Features, of, AW, Bellies, Sandals, Wed..."
3,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
4,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies,Sicons,"[[Specifications, of, Sicons, All, Purpose, Ar..."


In [66]:
new_df.isnull().sum()

Name_ID        0
mainsnak       0
brand       5864
TEXT           0
dtype: int64

In [67]:
new_df = new_df.fillna("unknown")

In [68]:
new_df.isnull().sum()

Name_ID     0
mainsnak    0
brand       0
TEXT        0
dtype: int64

In [69]:
new_df.head()

,Name_ID,mainsnak,brand,TEXT
0,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
1,FabHomeDecor Fabric Double Sofa Bed,Furniture,FabHomeDecor,"[[FabHomeDecor, Fabric, Double, Sofa, Bed, (, ..."
2,AW Bellies,Footwear,AW,"[[Key, Features, of, AW, Bellies, Sandals, Wed..."
3,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
4,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies,Sicons,"[[Specifications, of, Sicons, All, Purpose, Ar..."


create a function "add_back_nameID", which adds the nameID to the text column

In [70]:
# new_df.to_csv("/content/drive/MyDrive/test_data/flipkart_cleaned.csv", index=False)

In [71]:
# copy new_df in processed_df
processed_df = new_df.copy()

In [72]:
len(processed_df)

20000

In [73]:
processed_df.head()

,Name_ID,mainsnak,brand,TEXT
0,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
1,FabHomeDecor Fabric Double Sofa Bed,Furniture,FabHomeDecor,"[[FabHomeDecor, Fabric, Double, Sofa, Bed, (, ..."
2,AW Bellies,Footwear,AW,"[[Key, Features, of, AW, Bellies, Sandals, Wed..."
3,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
4,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies,Sicons,"[[Specifications, of, Sicons, All, Purpose, Ar..."


In [77]:
def data_preparation(df):
  # word_dict = {}
  data_list = []
  for index, row in df.iterrows():
    #text = [item for entry in row['TEXT'] for item in entry if item!=""]
    word_dict = {"Name_ID": row["Name_ID"], "taxon rank": [{"mainsnak": "Product"}],
                "parent taxon": [{"mainsnak": row["brand"]}], "instance of": [{"mainsnak": row['mainsnak']}], 
                "TEXT": row['TEXT']}
    data_list.append(word_dict.copy())
  return data_list

In [78]:
data_list_processed = data_preparation(processed_df)

In [79]:
# create a file name called new_df.json and write the data.
import json
with open("/content/drive/MyDrive/datasets/processed.json", "w") as f:
  for item in data_list_processed:
    f.write(json.dumps(item))
    f.write("\n")

In [80]:
# create another copy of new_df and further process it to add Name_ID back to the TEXT column.
addback_df = new_df.copy()

In [81]:
addback_df.head()

,Name_ID,mainsnak,brand,TEXT
0,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
1,FabHomeDecor Fabric Double Sofa Bed,Furniture,FabHomeDecor,"[[FabHomeDecor, Fabric, Double, Sofa, Bed, (, ..."
2,AW Bellies,Footwear,AW,"[[Key, Features, of, AW, Bellies, Sandals, Wed..."
3,Alisha Solid Womens Cycling Shorts,Clothing,Alisha,"[[Key, Features, of, Alisha, Solid, Womens, Cy..."
4,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies,Sicons,"[[Specifications, of, Sicons, All, Purpose, Ar..."


In [84]:
def add_back_nameID(df):
  
  product_df=df.copy()

  print(len(product_df))
  # new_list = []
  for index, row in product_df.iterrows():
    # print(row)
    first_list = row["TEXT"][0].copy()
    # print("This is first list in a row")
    print(first_list)
    # print(row["Name_ID"])
    name_list = row["Name_ID"].split()
    # print(name_list)
    join_token = name_list[0]
    name_length = len(name_list)
    # print("lenght of name_list={}".format(name_length))
    # print(join_token)
    if join_token in first_list:
      # print("hello")
      try:
        word_index = first_list.index(join_token)
        # print("word_index", word_index)
      except ValueError as e:
        print(e)
      else:
        # for i in range(word_index, name_length-1):
        del first_list[word_index:word_index+name_length]
        print("\n")
        # print(".......................")
        # print(first_list)
        first_list.insert(word_index, row["Name_ID"])
        print(first_list)
        print("\n")
        print(".......................")
        row["TEXT"][0] = first_list
        product_df.at[index, "TEXT"]=row["TEXT"]
        # new_list.append(first_list)
        # print(row["TEXT"])
  return product_df





In [85]:
product_df = add_back_nameID(addback_df)

Streaming output truncated to the last 5000 lines.


['Key', 'Features', 'of', 'American Club ExclusiveTrolleyBag4DT Expandable  Checkin Luggage  28', 'Luggage', '-', '28', 'Outer', 'Material', '-', 'Nylon/Teflon', 'Inner', 'Material', '-', 'Satin/Nylon', 'Compartment', '-', '2', 'Outside', '1', 'Inside', 'Shell', 'Type', '-', 'Soft', 'Top', 'Handle', 'Type', '-', 'Inbuilt', 'Lock', 'Type', '-', 'Combination', 'Lock', 'Water', 'Resistant', 'Number', 'Of', 'Wheels', '-', '4', 'Light', 'weight', 'Durable', 'Stylish', '&', 'Trendy', 'Look', '360', 'Degree', 'Rotation', 'High', 'Grade', 'Aluminium', 'Handle', 'Expandable', 'American', 'Club', 'ExclusiveTrolleyBag4DT', 'Expandable', 'Check', '-', 'in', 'Luggage', '-', '28', '(', 'Maroon', ')', 'Price', '.']


.......................
['Key', 'Features', 'of', 'PRINT', 'SHAPES', 'monster', 'university', 'party', 'Laptop', 'Skin', 'with', 'Mouse', 'pad', 'Combo', 'Set', 'Glue', 'Free', 'Removal', '15', '6', 'inches', 'Laptop', 'skin', 'Easy', 

In [86]:
data_list_product = data_preparation(product_df)

In [87]:
# create a file name called new_df.json and write the data.
import json
with open("/content/drive/MyDrive/datasets/flipkart_products.json", "w") as f:
  for item in data_list_product:
    f.write(json.dumps(item))
    f.write("\n")

trial version dataframe and check whether everything is working fine

In [ ]:
# sample_df = new_df[:20]

In [ ]:
# trial_df = sample_df.copy()

In [ ]:
# create a file name called new_df.json and write the data.
# import json
# with open("/content/drive/MyDrive/test_data/test_df.json", "w") as f:
#   for item in sample_list:
#     f.write(json.dumps(item))
#     f.write("\n")

In [ ]:
# test_df = sample_df.copy()

In [ ]:
# trial_df["TEXT"]

In [ ]:
# def test_df_trail(test_df):
  
#   new_list = []
#   for index, row in test_df.iterrows():
#     # print(row)
#     first_list = row["TEXT"][0].copy()
#     # print("This is first list in a row")
#     print("\n")
#     print(first_list)
#     print(row["Name_ID"])
#     name_list = row["Name_ID"].split()
#     print(name_list)
#     join_token = name_list[0]
#     name_length = len(name_list)
#     print("lenght of name_list={}".format(name_length))
#     print(join_token)
#     if join_token in first_list:
#       print("hello")
#       try:
#         word_index = first_list.index(join_token)
#         print("word_index", word_index)
#       except ValueError as e:
#         print(e)
#       else:
#         # for i in range(word_index, name_length-1):
#         del first_list[word_index:word_index+name_length]
#         print("\n")
#         print(".......................")
#         print(first_list)
#         first_list.insert(word_index, row["Name_ID"])
#         print(first_list)
#         print("\n")
#         row["TEXT"][0] = first_list
#         test_df.at[index, "TEXT"]=row["TEXT"]
#         # row["TEXT"][0] = first_list
#         # new_list.append(first_list)
#         print(row["TEXT"])
#   print("test_df")
#   return test_df





In [ ]:
# test_df_new = test_df_trail(trial_df)

In [ ]:
# new_test_df_list = data_preparation(test_df_new)

In [ ]:
# # create a file name called new_df.json and write the data.
# import json
# with open("/content/drive/MyDrive/test_data/new_test_df.json", "w") as f:
#   for item in new_test_df_list:
#     f.write(json.dumps(item))
#     f.write("\n")